In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

In [3]:
#Read Data
train_variant = pd.read_csv("training_variants")
test_variant = pd.read_csv("test_variants")
train_text = pd.read_csv("training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_text = pd.read_csv("test_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

In [4]:
train_variant.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [5]:
test_variant.head()

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A


In [6]:
train_text.head()

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [7]:
test_text.head()

,ID,Text
0,0,2. This mutation resulted in a myeloproliferat...
1,1,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,Vascular endothelial growth factor receptor (V...
3,3,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,Abstract Retinoblastoma is a pediatric retina...


In [8]:
#Train data number of observations in each class
##Class1:568#Class2:452#Class3:89#Class4:686
##Class6:275#Class7:953#Class8:19#Class9:37
##Class5:242
train = pd.merge(train_variant, train_text, how='left', on='ID')
x_train = train.drop('Class', axis=1)

In [9]:
x_train.head()

,ID,Gene,Variation,Text
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...


In [10]:
x_test = pd.merge(test_variant, test_text, how='left', on='ID')
test_index = x_test['ID'].values
x_test.head()

,ID,Gene,Variation,Text
0,0,ACSL4,R570S,2. This mutation resulted in a myeloproliferat...
1,1,NAGLU,P521L,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,PAH,L333F,Vascular endothelial growth factor receptor (V...
3,3,ING1,A148D,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,TMEM216,G77A,Abstract Retinoblastoma is a pediatric retina...


In [11]:
data = np.concatenate((x_train, x_test), axis=0)
data=pd.DataFrame(data)
data.columns = ["ID", "Gene", "Variation", "Text"]
data.head()

,ID,Gene,Variation,Text
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...


In [12]:
data.shape

(8989, 4)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
#TFIDF
mod=TfidfVectorizer(min_df=5, max_features=500, stop_words='english')
mod_TD=mod.fit_transform(data.Text)

In [15]:
from sklearn.decomposition import TruncatedSVD

In [16]:
#SVD features
SVD=TruncatedSVD(200,random_state=41)
SVD_FIT=SVD.fit_transform(mod_TD)
yet_to_complete=pd.DataFrame(SVD_FIT)

In [17]:
from sklearn import preprocessing

In [18]:
#data.drop(data.columns[[0,3]],inplace=True, axis=1)
#as Gene and Variation data values are just scattered like IDS, i dont think these give u great info about the prediction
encoder = preprocessing.LabelEncoder()
y_train = train['Class'].values
encoder.fit(y_train)
encoded_y = encoder.transform(y_train)

In [19]:
y_train

array([1, 2, 2, ..., 1, 4, 4], dtype=int64)

In [20]:
encoded_y

array([0, 1, 1, ..., 0, 3, 3], dtype=int64)

In [21]:
X_Train=yet_to_complete[:2700]
Y_Train=encoded_y[:2700]
X_Test=yet_to_complete[2700:3321]
Y_Test=encoded_y[2700:3321]

Tune XGboost

1 Tune max_depth and min_child_weight

In [23]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import cross_validation, metrics   #Additional scklearn functions
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import log_loss
import time

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [75]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import cross_validation, metrics   #Additional scklearn functions
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import log_loss
import time
start = time.time()

param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
grid = GridSearchCV(estimator= XGBClassifier( learning_rate =0.1, n_estimators=50, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)

grid.fit(X_Train,Y_Train)

print(grid.grid_scores_)
print(grid.best_params_)
print(grid.best_score_)

#grid = grid.fit(X_train, Y_train)
model=grid.best_estimator_

y_pred = model.predict_proba(X_Test)
print(log_loss( Y_Test,y_pred))

print(time.time() - start) 

[mean: -1.64314, std: 0.12174, params: {'max_depth': 3, 'min_child_weight': 1}, mean: -1.63337, std: 0.11299, params: {'max_depth': 3, 'min_child_weight': 3}, mean: -1.63167, std: 0.10955, params: {'max_depth': 3, 'min_child_weight': 5}, mean: -1.75748, std: 0.16160, params: {'max_depth': 5, 'min_child_weight': 1}, mean: -1.73040, std: 0.13626, params: {'max_depth': 5, 'min_child_weight': 3}, mean: -1.72278, std: 0.13731, params: {'max_depth': 5, 'min_child_weight': 5}, mean: -1.87715, std: 0.18842, params: {'max_depth': 7, 'min_child_weight': 1}, mean: -1.81128, std: 0.17085, params: {'max_depth': 7, 'min_child_weight': 3}, mean: -1.77982, std: 0.15380, params: {'max_depth': 7, 'min_child_weight': 5}, mean: -1.91990, std: 0.20536, params: {'max_depth': 9, 'min_child_weight': 1}, mean: -1.85560, std: 0.16665, params: {'max_depth': 9, 'min_child_weight': 3}, mean: -1.81193, std: 0.15868, params: {'max_depth': 9, 'min_child_weight': 5}]
{'max_depth': 3, 'min_child_weight': 5}
-1.63167012

C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [76]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import cross_validation, metrics   #Additional scklearn functions
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import log_loss
import time
start = time.time()

param_test1 = {
 'max_depth':[1,2,3,4],
 'min_child_weight':[4,5,6,7]
}
grid = GridSearchCV(estimator= XGBClassifier( learning_rate =0.1, n_estimators=50, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)

grid.fit(X_Train,Y_Train)

print(grid.grid_scores_)
print(grid.best_params_)
print(grid.best_score_)

#grid = grid.fit(X_train, Y_train)
model=grid.best_estimator_

y_pred = model.predict_proba(X_Test)
print(log_loss( Y_Test,y_pred))

print(time.time() - start) 

[mean: -1.61567, std: 0.07891, params: {'max_depth': 1, 'min_child_weight': 4}, mean: -1.61258, std: 0.08158, params: {'max_depth': 1, 'min_child_weight': 5}, mean: -1.61235, std: 0.08368, params: {'max_depth': 1, 'min_child_weight': 6}, mean: -1.61087, std: 0.08901, params: {'max_depth': 1, 'min_child_weight': 7}, mean: -1.60002, std: 0.09908, params: {'max_depth': 2, 'min_child_weight': 4}, mean: -1.60069, std: 0.09626, params: {'max_depth': 2, 'min_child_weight': 5}, mean: -1.59768, std: 0.10106, params: {'max_depth': 2, 'min_child_weight': 6}, mean: -1.59514, std: 0.10499, params: {'max_depth': 2, 'min_child_weight': 7}, mean: -1.63326, std: 0.11301, params: {'max_depth': 3, 'min_child_weight': 4}, mean: -1.63167, std: 0.10955, params: {'max_depth': 3, 'min_child_weight': 5}, mean: -1.63426, std: 0.11336, params: {'max_depth': 3, 'min_child_weight': 6}, mean: -1.62647, std: 0.11988, params: {'max_depth': 3, 'min_child_weight': 7}, mean: -1.67529, std: 0.12781, params: {'max_depth':

C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [77]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import cross_validation, metrics   #Additional scklearn functions
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import log_loss
import time
start = time.time()

param_test1 = {
 'max_depth':[1,2,3,],
 'min_child_weight':[6,7,8,9,10]
}
grid = GridSearchCV(estimator= XGBClassifier( learning_rate =0.1, n_estimators=50, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)

grid.fit(X_Train,Y_Train)

print(grid.grid_scores_)
print(grid.best_params_)
print(grid.best_score_)

#grid = grid.fit(X_train, Y_train)
model=grid.best_estimator_

y_pred = model.predict_proba(X_Test)
print(log_loss( Y_Test,y_pred))

print(time.time() - start) 

[mean: -1.61235, std: 0.08368, params: {'max_depth': 1, 'min_child_weight': 6}, mean: -1.61087, std: 0.08901, params: {'max_depth': 1, 'min_child_weight': 7}, mean: -1.60899, std: 0.09019, params: {'max_depth': 1, 'min_child_weight': 8}, mean: -1.60874, std: 0.09000, params: {'max_depth': 1, 'min_child_weight': 9}, mean: -1.60696, std: 0.08874, params: {'max_depth': 1, 'min_child_weight': 10}, mean: -1.59768, std: 0.10106, params: {'max_depth': 2, 'min_child_weight': 6}, mean: -1.59514, std: 0.10499, params: {'max_depth': 2, 'min_child_weight': 7}, mean: -1.58845, std: 0.10693, params: {'max_depth': 2, 'min_child_weight': 8}, mean: -1.58952, std: 0.11059, params: {'max_depth': 2, 'min_child_weight': 9}, mean: -1.59129, std: 0.10090, params: {'max_depth': 2, 'min_child_weight': 10}, mean: -1.63426, std: 0.11336, params: {'max_depth': 3, 'min_child_weight': 6}, mean: -1.62647, std: 0.11988, params: {'max_depth': 3, 'min_child_weight': 7}, mean: -1.61711, std: 0.12094, params: {'max_depth

C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


2.3 Tune gamma

In [78]:
start = time.time()

param_test1 = {
 'gamma':[0.0,0.1,0.2,0.3,0.4,0.5]
}
grid = GridSearchCV(estimator= XGBClassifier( learning_rate =0.1, n_estimators=50, max_depth=2,
 min_child_weight=8, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)

grid.fit(X_Train,Y_Train)

print(grid.grid_scores_)
print(grid.best_params_)
print(grid.best_score_)

#grid = grid.fit(X_train, Y_train)
model=grid.best_estimator_

y_pred = model.predict_proba(X_Test)
print(log_loss( Y_Test,y_pred))

print(time.time() - start) 

[mean: -1.58845, std: 0.10693, params: {'gamma': 0.0}, mean: -1.58843, std: 0.10691, params: {'gamma': 0.1}, mean: -1.58862, std: 0.10699, params: {'gamma': 0.2}, mean: -1.58934, std: 0.10617, params: {'gamma': 0.3}, mean: -1.58925, std: 0.10530, params: {'gamma': 0.4}, mean: -1.59081, std: 0.10434, params: {'gamma': 0.5}]
{'gamma': 0.1}
-1.58843051155
1.63848116298
158.0340392589569


C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


2.4 Tune subsample and colsample_bytree

In [80]:
start = time.time()

param_test1 = {
 'subsample':[0.6,0.7,0.8,0.9,1.0], 'colsample_bytree':[0.6,0.7,0.8,0.9,1.0]
}
grid = GridSearchCV(estimator= XGBClassifier( learning_rate =0.1, n_estimators=50, max_depth=2,
 min_child_weight=8, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)

grid.fit(X_Train,Y_Train)

print(grid.grid_scores_)
print(grid.best_params_)
print(grid.best_score_)

#grid = grid.fit(X_train, Y_train)
model=grid.best_estimator_

y_pred = model.predict_proba(X_Test)
print(log_loss( Y_Test,y_pred))

print(time.time() - start) 

[mean: -1.58848, std: 0.09991, params: {'colsample_bytree': 0.6, 'subsample': 0.6}, mean: -1.58324, std: 0.11041, params: {'colsample_bytree': 0.6, 'subsample': 0.7}, mean: -1.58605, std: 0.10730, params: {'colsample_bytree': 0.6, 'subsample': 0.8}, mean: -1.59388, std: 0.10873, params: {'colsample_bytree': 0.6, 'subsample': 0.9}, mean: -1.59900, std: 0.11262, params: {'colsample_bytree': 0.6, 'subsample': 1.0}, mean: -1.60113, std: 0.10677, params: {'colsample_bytree': 0.7, 'subsample': 0.6}, mean: -1.59113, std: 0.10825, params: {'colsample_bytree': 0.7, 'subsample': 0.7}, mean: -1.58765, std: 0.10773, params: {'colsample_bytree': 0.7, 'subsample': 0.8}, mean: -1.59544, std: 0.10831, params: {'colsample_bytree': 0.7, 'subsample': 0.9}, mean: -1.59707, std: 0.10863, params: {'colsample_bytree': 0.7, 'subsample': 1.0}, mean: -1.59482, std: 0.10293, params: {'colsample_bytree': 0.8, 'subsample': 0.6}, mean: -1.59015, std: 0.10752, params: {'colsample_bytree': 0.8, 'subsample': 0.7}, mea

C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


2.5 Tune Tuning Regularization Parameters

In [81]:
start = time.time()

param_test1 = {
 'reg_alpha':[0,0.0001,0.001, 0.01, 0.1, 1,10, 100]
}
grid = GridSearchCV(estimator= XGBClassifier( learning_rate =0.1, n_estimators=50, max_depth=2,
 min_child_weight=8, gamma=0.1, subsample=0.9, colsample_bytree=0.9,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)

grid.fit(X_Train,Y_Train)

print(grid.grid_scores_)
print(grid.best_params_)
print(grid.best_score_)

#grid = grid.fit(X_train, Y_train)
model=grid.best_estimator_

y_pred = model.predict_proba(X_Test)
print(log_loss( Y_Test,y_pred))

print(time.time() - start) 

[mean: -1.58182, std: 0.10459, params: {'reg_alpha': 0}, mean: -1.58182, std: 0.10459, params: {'reg_alpha': 0.0001}, mean: -1.58162, std: 0.10430, params: {'reg_alpha': 0.001}, mean: -1.58227, std: 0.10262, params: {'reg_alpha': 0.01}, mean: -1.58682, std: 0.10517, params: {'reg_alpha': 0.1}, mean: -1.58608, std: 0.10626, params: {'reg_alpha': 1}, mean: -1.58982, std: 0.09959, params: {'reg_alpha': 10}, mean: -1.81109, std: 0.03861, params: {'reg_alpha': 100}]
{'reg_alpha': 0.001}
-1.58161616692
1.63986522617
191.71596550941467


C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


2.6 Reducing Learning Rate re-calibrate the number of boosting rounds

In [82]:
start = time.time()

param_test1 = {
 'n_estimators':[50,100,300,500,700,1000,2000,3000,4000,5000]
}
grid = GridSearchCV(estimator= XGBClassifier( learning_rate =0.01, n_estimators=50, max_depth=2,
 min_child_weight=8, gamma=0.1, subsample=0.9, colsample_bytree=0.9,reg_alpha=0.001,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)

grid.fit(X_Train,Y_Train)

print(grid.grid_scores_)
print(grid.best_params_)
print(grid.best_score_)

#grid = grid.fit(X_train, Y_train)
model=grid.best_estimator_

y_pred = model.predict_proba(X_Test)
print(log_loss( Y_Test,y_pred))

print(time.time() - start) 

[mean: -1.93679, std: 0.03969, params: {'n_estimators': 50}, mean: -1.79973, std: 0.06082, params: {'n_estimators': 100}, mean: -1.61490, std: 0.09063, params: {'n_estimators': 300}, mean: -1.58443, std: 0.10415, params: {'n_estimators': 500}, mean: -1.58770, std: 0.11590, params: {'n_estimators': 700}, mean: -1.61258, std: 0.12722, params: {'n_estimators': 1000}, mean: -1.70785, std: 0.15774, params: {'n_estimators': 2000}, mean: -1.80898, std: 0.18192, params: {'n_estimators': 3000}, mean: -1.90401, std: 0.20212, params: {'n_estimators': 4000}, mean: -1.99316, std: 0.22057, params: {'n_estimators': 5000}]
{'n_estimators': 500}
-1.58443174497
1.6187840573
7373.850759983063


C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [39]:
model=XGBClassifier( learning_rate =0.01, n_estimators=500, max_depth=2,
 min_child_weight=8, gamma=0.1, subsample=0.9, colsample_bytree=0.9,reg_alpha=0.001,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27)

In [36]:
model.fit(yet_to_complete[:3321],encoded_y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0.1, learning_rate=0.01, max_delta_step=0, max_depth=2,
       min_child_weight=8, missing=None, n_estimators=500, nthread=4,
       objective='multi:softprob', reg_alpha=0.001, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.9)

In [89]:
#predictions
y_pred=gbm1.predict_proba(yet_to_complete[3321:])

In [38]:
#tweaking the submission file as required
subm_file = pd.DataFrame(y_pred)
subm_file['id'] = test_index
subm_file.columns = ['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8', 'class9', 'id']
subm_file.to_csv("submission.csv",index=False)

scored 1.82637